In [1]:
import pandas as pd
import yfinance as yf

# Define the ticker symbol for the S&P 500 index
ticker = '^GSPC'

# Set the date range
start_date = '2008-01-01'
end_date = '2023-12-31'

# Download the historical OHLCV data
sp500_data = yf.download(ticker, start=start_date, end=end_date)

C:\Users\yagne\AppData\Local\Temp\ipykernel_5248\1808062044.py:12: FutureWarning: YF.download() has changed argument auto_adjust default to True
  sp500_data = yf.download(ticker, start=start_date, end=end_date)
[*********************100%***********************]  1 of 1 completed


In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

# Load tokenizer and model from Hugging Face
tokenizer = AutoTokenizer.from_pretrained("yiyanghkust/finbert-tone")
model = AutoModelForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone")

# Create sentiment analysis pipeline
finbert = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


Device set to use cpu


In [4]:
headline = "Tesla reports record profits beating analyst expectations"

result = finbert(headline)
print(result)


[{'label': 'Positive', 'score': 1.0}]


In [5]:
dn = pd.read_csv("news_data.csv")
print(dn.head(10))
df = pd.read_csv("ohlcv_data.csv")

                                               Title       Date       CP
0   JPMorgan Predicts 2008 Will Be "Nothing But Net"   1/2/2008  1447.16
1  Dow Tallies Biggest First-session-of-year Poin...   1/2/2008  1447.16
2                   2008 predictions for the S&P 500   1/2/2008  1447.16
3  U.S. Stocks Higher After Economic Data, Monsan...   1/3/2008  1447.16
4  U.S. Stocks Climb As Hopes Increase For More F...   1/7/2008  1416.18
5  How Investing in Intangibles -- Like Employee ...   1/9/2008  1409.13
6         Head And Shoulders Top Bodes Ill For Bulls   1/9/2008  1409.13
7  U.S. Stocks Zigzag Higher As Bernanke Speech S...  1/10/2008  1420.33
8  It's a Black Monday as stock markets tank in e...  1/22/2008  1310.50
9      U.S. Stocks Largely Recover From Early Plunge  1/22/2008  1310.50


In [6]:
for i in dn['Title'].head(10):
    results = finbert(i)
    print(results)

[{'label': 'Neutral', 'score': 0.9923882484436035}]
[{'label': 'Positive', 'score': 0.4879378378391266}]
[{'label': 'Neutral', 'score': 0.999624490737915}]
[{'label': 'Positive', 'score': 0.9999986886978149}]
[{'label': 'Positive', 'score': 0.9838525056838989}]
[{'label': 'Neutral', 'score': 0.9983394145965576}]
[{'label': 'Negative', 'score': 0.9846346974372864}]
[{'label': 'Neutral', 'score': 0.8774247765541077}]
[{'label': 'Neutral', 'score': 0.9999880790710449}]
[{'label': 'Positive', 'score': 0.998782217502594}]


In [15]:
from pandas import pivot_table


sentiments = []
sentiment_score = []

for h in dn['Title']:
    result = finbert(h)[0]
    sentiments.append(result['label'])
    sentiment_score.append(round(result['score'],2))

dn['sentiment_score'] = sentiment_score
dn['sentiment'] = sentiments

# print(dn.head(11))

# Mean sentiment score per day
daily_mean = groupby("Date", sort= False)["sentiment_score"].mean().round(2).reset_index(name="mean_sentiment")

# Mean per sentiment category
pivot = pivot_table(values="sentiment_score", index="Date", columns="sentiment", aggfunc="mean", sort= False)

# Merge them
result = daily_mean.merge(pivot, on="Date", how="left").fillna(0)

# print(result)
# Save to CSV
result.to_csv("processed_data.csv", index=False)


NameError: name 'groupby' is not defined

In [21]:
dn.iloc[0:11].groupby('Date', sort= False)['sentiment_score'].mean().round(2)


Date
1/2/2008     0.83
1/3/2008     1.00
1/7/2008     0.98
1/9/2008     0.99
1/10/2008    0.88
1/22/2008    1.00
Name: sentiment_score, dtype: float64

In [22]:
# dm = pd.merge(dn, df , how="right" , on="Date")
# dm.fillna(0)
# dm.head(20)